In [ ]:
#Only run in colab
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Github/Taskmaster
#!pip install tensorflow --upgrade
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

In [1]:
import rubiks as rubiks
import numpy as np
from student import student
from classroom import classroom
from teacher import teacher
from rubiks_network_experimental import student_network
from utils import dotdict
from keras import regularizers

task = rubiks.rubiks_task
setup = rubiks.rubiks_setup
rng = np.random.default_rng(seed=0)

core_params = dotdict({
            "residual_weights_reg" : None,
            "residual_bias_reg" : None,
            "relu_leak" : 0.05,
            "residual_units" : 512,
            "learning_rate" : 0.001,
            "residual_layers" : 2
})

value_network_params = dotdict({
            "residual_weights_reg" : None,
            "residual_bias_reg" : None,
            "relu_leak" : 0.05,
            "residual_units" : 512,
            "learning_rate" : 0.001,
            "post_core_residual_layers" : 2,
            "reward_fork_layers": 1,
            "value_fork_layers": 1
})

state_network_params = dotdict({
            "residual_weights_reg" : None,
            "residual_bias_reg" : None,
            "relu_leak" : 0.05,
            "residual_units" : 512,
            "learning_rate" : 0.001,
            "post_core_residual_layers" : 2
})

params = dotdict({
    "core_params":core_params,
    "value_network_params": value_network_params,
    "state_network_params": state_network_params,
    "state_size":task.input_size,
    "action_codes": task.n_actions
})

my_student_network = student_network.create(params)
#my_student_network = student_network.load("models/combined/trained/trained")
student_template = student(task, 50,my_student_network,0.95)
t = teacher(setup, lambda n : 1+rng.poisson(lam=5,size=n))
c = classroom(task, setup, t, student_template, n_students=1, max_steps=10, buffer_size = lambda n : np.minimum(4096,np.maximum(1024,n//2)))

In [2]:
c.max_steps=10
c.buffer_size = lambda n : np.minimum(4096,np.maximum(1024,n//2))
t.step_dist = lambda n : 5*np.ones(n,dtype=int)

In [3]:

for i in range(50):
    print(f"Round {i}.")
    c.run_training_batch(n_problems=100,epochs_per_episode=3)
    #if i % 10 == 9:
    #    my_student_network.save("models/trained.h5")

Round 0.
Before step 1, 100 out of 100 remain open.
Before step 2, 100 out of 100 remain open.
Before step 3, 99 out of 100 remain open.
Before step 4, 99 out of 100 remain open.
Before step 5, 98 out of 100 remain open.
Before step 6, 98 out of 100 remain open.
Before step 7, 98 out of 100 remain open.
Before step 8, 98 out of 100 remain open.


KeyboardInterrupt: 

In [11]:
n=8192
rng = np.random.default_rng()
states = np.arange(task.input_size)[None,:].repeat(n,axis=0)

for k in range(80):
    print(f"Round {k}.")
    #c.run_training_batch(n_problems=100,epochs_per_episode=1)

    for i in range(50):
        _, states = rubiks.task_action(states,rng.choice(a=task.n_actions,size=n))

    actions = rng.choice(a=task.n_actions,size=n)
    _, next_states = rubiks.task_action(states,actions)

    actions = (np.arange(task.n_actions) == actions[...,None]).astype(float)
    colors = rubiks.make_neural_input(states)
    next_colors = rubiks.make_neural_input(next_states)
    my_student_network.fit_state(colors,actions,next_colors.reshape(n,54,6),epochs = 1)

    #if k % 10 == 9:
    #  my_student_network.save("models/combined/trained")

Round 0.
120/120 [==============================] - 4s 35ms/step - loss: 0.0262 - accuracy: 0.9909 - val_loss: 0.0262 - val_accuracy: 0.9912
Round 1.
120/120 [==============================] - 4s 34ms/step - loss: 0.0264 - accuracy: 0.9908 - val_loss: 0.0279 - val_accuracy: 0.9904
Round 2.
120/120 [==============================] - 4s 33ms/step - loss: 0.0250 - accuracy: 0.9913 - val_loss: 0.0264 - val_accuracy: 0.9907
Round 3.
120/120 [==============================] - 4s 34ms/step - loss: 0.0242 - accuracy: 0.9916 - val_loss: 0.0240 - val_accuracy: 0.9913
Round 4.
120/120 [==============================] - 4s 33ms/step - loss: 0.0246 - accuracy: 0.9914 - val_loss: 0.0245 - val_accuracy: 0.9911
Round 5.
120/120 [==============================] - 4s 33ms/step - loss: 0.0238 - accuracy: 0.9917 - val_loss: 0.0248 - val_accuracy: 0.9910
Round 6.
120/120 [==============================] - 4s 34ms/step - loss: 0.0237 - accuracy: 0.9917 - val_loss: 0.0234 - val_accuracy: 0.9912
Round 7.
120/

In [ ]:
from rubiks import print_coloring
states =  np.arange(54,dtype=int)[None,:]
for i in range(50):
    _, states = rubiks.task_action(states,rng.choice(a=task.n_actions,size=1))
actions = np.array([9],dtype=int)
prediction = my_student_network.predict_state(task.make_input(states),(np.arange(task.n_actions) == actions[...,None]).astype(float))
print_coloring(prediction.reshape(54,6).argmax(axis=-1),color_array=True)
print_coloring(rubiks.task_action(states,actions)[1][0])

In [12]:
n=8192
rng = np.random.default_rng()
states = np.arange(task.input_size)[None,:].repeat(n,axis=0)

for i in range(50):
    _, states = rubiks.task_action(states,rng.choice(a=task.n_actions,size=n))

actions = rng.choice(a=task.n_actions,size=n)
_, next_states = rubiks.task_action(states,actions)

actions = (np.arange(task.n_actions) == actions[...,None]).astype(float)
colors = rubiks.make_neural_input(states)
next_colors = rubiks.make_neural_input(next_states)
results = my_student_network.predict_state(colors,actions).argmax(axis=-1)

same = np.all(next_colors.reshape(n,54,6).argmax(axis=-1) == results,axis=-1)
same.sum()/same.size

256/256 [==============================] - 2s 7ms/step


0.776611328125